
#  Сбор данных

Собираем данные [о ценах на книги.](http://books.toscrape.com)


In [37]:
from typing import List
import warnings
warnings.filterwarnings("ignore")


In [38]:
from requests import get
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [39]:
import time

In [40]:
!pip3 install lxml

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [41]:
# фиктивная проверка


# 1. Сбор ссылок на книги

__Пишем функцию `get_soup`, которая по ссылке возвращает html-разметку страницы в формате `bs4`__

In [42]:
def get_page_soup(url_link: str) -> BeautifulSoup:
    response = get(url_link)
    return BeautifulSoup(response.content, 'html.parser')

In [43]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 5

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:31" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link 

In [44]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 5

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup.find_all('a')[0].text

'Books to Scrape'

__Пишем функцию `get_books_links`, которая находит в html-разметке страницы ссылки на странички с отдельными книгами.__

In [ ]:
def get_book_links(page_soup: BeautifulSoup) -> List[str]:
    books = soup.find_all('li', {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    return [item.h3.a.get('href') for item in books]

In [32]:
books = soup.find_all('li', {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'}) 
books

[<li class="col-xs-6 col-sm-4 col-md-3 col-lg-3">
 <article class="product_pod">
 <div class="image_container">
 <a href="princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html"><img alt="Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Princess Jellyfish 2-in-1 Omnibus #1)" class="thumbnail" src="../media/cache/78/0b/780b2c28122750c2c383846155815bf7.jpg"/></a>
 </div>
 <p class="star-rating Five">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>
 <h3><a href="princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html" title="Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Princess Jellyfish 2-in-1 Omnibus #1)">Princess Jellyfish 2-in-1 Omnibus, ...</a></h3>
 <div class="product_price">
 <p class="price_color">£13.61</p>
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>
 <form>
 <button clas

In [68]:
books[0]

'princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html'

In [36]:
books[0].h3.a.get('href')

'princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html'

In [55]:
books = soup.find_all('li', {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
books[0].h3.a.get('href')

'princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html'

__С помощью цикла собераем в лист `book_links` первые 200 книг.__

In [66]:
from tqdm.notebook import tqdm
import time

book_links = []
main_url = 'https://books.toscrape.com/catalogue/category/books_1/'



for p in tqdm(range(1, 11)):
    url = main_url + f'page-{p}.html'
    page_soup = get_page_soup(url)
    books = get_book_links(page_soup)
    book_links.extend(books)
    time.sleep(0.02)



  0%|          | 0/10 [00:00<?, ?it/s]

In [90]:
len(book_links)

200

# 2. Сбор информации о книгах 

Пишем несколько небольших функций, которые собирают различные данные об одной книге. Информацию о книге собираем в виде словаря вида 

```
{ 'name': 'Преступление и наказание', 'rating': 1, 'description': 'ужасно депрессивная книга', ... }

```

- название
- рейтинг
- описание
- налог
- цена

Подгруждаем библиотеки

In [71]:
import requests
from bs4 import BeautifulSoup
import typing as tp
from tqdm.notebook import tqdm
import time

Используя ссылку на сайт, выгружаем с него контент и представляем его в виде дерева

In [73]:
def get_soup(url: str) -> BeautifulSoup:
    res = get(url)
    return BeautifulSoup(res.content, 'html.parser')

tree = get_soup('https://books.toscrape.com/catalogue/category/books_1/')
tree


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    Books | 
     Books to Scrape - Sandbox

</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    
" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../../../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../../../static/oscar/css/styles.css" rel="stylesheet" type="

Достаем часть информации (название, ссылку и рейтинг)

In [81]:
# рейтинг
books = tree.find_all('li',  {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
books[0].p.get('class')[1]

'Three'

In [85]:
def get_main_hrefs(tree: BeautifulSoup) -> tp.List[tp.Dict]:
    books = tree.find_all('li',  {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    return [{
        'link': item.h3.a.get('href'),
        'name': item.h3.a.get('title'),
        'rating': item.p.get('class')[1]
    } for item in books]

Применяем эти функции, чтобы пройтись цикломи по первым 200 книгам (первые 10 страниц) и вытащить информацию

In [86]:
MAIN_URL = 'https://books.toscrape.com/catalogue/category/books_1/'
data = [ ]

for p in tqdm(range(1, 11)):
    url = MAIN_URL + f'page-{p}.html'
    tree = get_soup(url)
    books = get_main_hrefs(tree)
    data.extend(books)
    time.sleep(0.02)

  0%|          | 0/10 [00:00<?, ?it/s]

In [87]:
data

[{'link': '../../a-light-in-the-attic_1000/index.html',
  'name': 'A Light in the Attic',
  'rating': 'Three'},
 {'link': '../../tipping-the-velvet_999/index.html',
  'name': 'Tipping the Velvet',
  'rating': 'One'},
 {'link': '../../soumission_998/index.html',
  'name': 'Soumission',
  'rating': 'One'},
 {'link': '../../sharp-objects_997/index.html',
  'name': 'Sharp Objects',
  'rating': 'Four'},
 {'link': '../../sapiens-a-brief-history-of-humankind_996/index.html',
  'name': 'Sapiens: A Brief History of Humankind',
  'rating': 'Five'},
 {'link': '../../the-requiem-red_995/index.html',
  'name': 'The Requiem Red',
  'rating': 'One'},
 {'link': '../../the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
  'name': 'The Dirty Little Secrets of Getting Your Dream Job',
  'rating': 'Four'},
 {'link': '../../the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
  'name': 'The Coming Woman: A Novel Based on the Life of the Infa

In [88]:
df = pd.DataFrame(data)
df


,link,name,rating
0,../../a-light-in-the-attic_1000/index.html,A Light in the Attic,Three
1,../../tipping-the-velvet_999/index.html,Tipping the Velvet,One
2,../../soumission_998/index.html,Soumission,One
3,../../sharp-objects_997/index.html,Sharp Objects,Four
4,../../sapiens-a-brief-history-of-humankind_996...,Sapiens: A Brief History of Humankind,Five
...,...,...,...
195,../../eureka-trivia-60_805/index.html,Eureka Trivia 6.0,Four
196,../../drive-the-surprising-truth-about-what-mo...,Drive: The Surprising Truth About What Motivat...,Four
197,../../done-rubbed-out-reightman-bailey-1_803/i...,Done Rubbed Out (Reightman & Bailey #1),Five
198,../../doing-it-over-most-likely-to-1_802/index...,Doing It Over (Most Likely To #1),Three


Пройдем циклом по всем сыслкам из списка `book_links` и соберем данные о книгах в вектор `book_info`. 

In [89]:
# список ссылок на книги
book_links

['princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html',
 'princess-between-worlds-wide-awake-princess-5_919/index.html',
 'pop-gun-war-volume-1-gift_918/index.html',
 'political-suicide-missteps-peccadilloes-bad-calls-backroom-hijinx-sordid-pasts-rotten-breaks-and-just-plain-dumb-mistakes-in-the-annals-of-american-politics_917/index.html',
 'patience_916/index.html',
 'outcast-vol-1-a-darkness-surrounds-him-outcast-1_915/index.html',
 'orange-the-complete-collection-1-orange-the-complete-collection-1_914/index.html',
 'online-marketing-for-busy-authors-a-step-by-step-guide_913/index.html',
 'on-a-midnight-clear_912/index.html',
 'obsidian-lux-1_911/index.html',
 'my-paris-kitchen-recipes-and-stories_910/index.html',
 'masks-and-shadows_909/index.html',
 'mama-tried-traditional-italian-cooking-for-the-screwed-crude-vegan-and-tattooed_908/index.html',
 'lumberjanes-vol-2-friendship-to-the-max-lumberjanes-5-8_907/index.html',
 'lumberjanes-vol-1-bew

{'link': '../../a-light-in-the-attic_1000/index.html', 'name': 'A Light in the Attic', 'rating': 'Three'}
{'link': '../../tipping-the-velvet_999/index.html', 'name': 'Tipping the Velvet', 'rating': 'One', 'UPC': '90fa61229261140a', 'Product Type': 'Books', 'Price (excl. tax)': '£53.74', 'Price (incl. tax)': '£53.74', 'Tax': '£0.00', 'Availability': 'In stock (20 available)', 'Number of reviews': '0', 'price': '£53.74'}
{'link': '../../soumission_998/index.html', 'name': 'Soumission', 'rating': 'One'}
{'link': '../../sharp-objects_997/index.html', 'name': 'Sharp Objects', 'rating': 'Four'}
{'link': '../../sapiens-a-brief-history-of-humankind_996/index.html', 'name': 'Sapiens: A Brief History of Humankind', 'rating': 'Five'}
{'link': '../../the-requiem-red_995/index.html', 'name': 'The Requiem Red', 'rating': 'One'}
{'link': '../../the-dirty-little-secrets-of-getting-your-dream-job_994/index.html', 'name': 'The Dirty Little Secrets of Getting Your Dream Job', 'rating': 'Four'}
{'link': '

In [165]:

href = MAIN_URL + data[1]['link'] # из него достается ссылка
tree = get_soup(href) # по ней что-то скачивается
description = tree.find('p', {'class': ''})
description

<p>"Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty's "Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty's dresser and the two head for the bright lights of Leicester Square where they begin a glittering career as music-hall stars in an all-singing and dancing double act. At the same time, behind closed doors, they admit their attraction to each other 

In [166]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

book_info = []

def get_book_info(infa): # на вход идет словарь
    href = MAIN_URL + infa['link'] # из него достается ссылка
    tree = get_soup(href) # по ней что-то скачивается
    table = tree.find('table', {'class': 'table table-striped'})
    table = pd.read_html(str(table))[0].set_index(0).to_dict()[1] 
    price = tree.find('p', {'class': 'price_color'}).text
    description = tree.find('p', {'class': ''})
    infa.update(table) # словарь, который был на входе, обновляется, на выход выскакивает
    infa.update({'price': price})
    infa.update({'description': description})
    return infa

# your code here


In [167]:
updated_data = [ ]

for i in range(0, 200):
    try:
        updated_data.append(get_book_info(data[i]))
    except:
        print('Не получилось', item['href'])

In [168]:
dff = pd.DataFrame(updated_data)
dff.head()

,link,name,rating,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews,price,description
0,../../a-light-in-the-attic_1000/index.html,A Light in the Attic,Three,a897fe39b1053632,Books,£51.77,£51.77,£0.00,In stock (22 available),0,£51.77,[It's hard to imagine a world without A Light ...
1,../../tipping-the-velvet_999/index.html,Tipping the Velvet,One,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74,"[""Erotic and absorbing...Written with starling..."
2,../../soumission_998/index.html,Soumission,One,6957f44c3847a760,Books,£50.10,£50.10,£0.00,In stock (20 available),0,£50.10,"[Dans une France assez proche de la nôtre, un ..."
3,../../sharp-objects_997/index.html,Sharp Objects,Four,e00eb4fd7b871a48,Books,£47.82,£47.82,£0.00,In stock (20 available),0,£47.82,"[WICKED above her hipbone, GIRL across her hea..."
4,../../sapiens-a-brief-history-of-humankind_996...,Sapiens: A Brief History of Humankind,Five,4165285e1663650f,Books,£54.23,£54.23,£0.00,In stock (20 available),0,£54.23,[From a renowned historian comes a groundbreak...


Превратим вектор из информации в полноценную таблицу с данными. 

In [169]:
import copy
book_info_df = copy.deepcopy(dff)
print(book_info_df.shape)
book_info_df.head() 

(200, 12)


,link,name,rating,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews,price,description
0,../../a-light-in-the-attic_1000/index.html,A Light in the Attic,Three,a897fe39b1053632,Books,£51.77,£51.77,£0.00,In stock (22 available),0,£51.77,[It's hard to imagine a world without A Light ...
1,../../tipping-the-velvet_999/index.html,Tipping the Velvet,One,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74,"[""Erotic and absorbing...Written with starling..."
2,../../soumission_998/index.html,Soumission,One,6957f44c3847a760,Books,£50.10,£50.10,£0.00,In stock (20 available),0,£50.10,"[Dans une France assez proche de la nôtre, un ..."
3,../../sharp-objects_997/index.html,Sharp Objects,Four,e00eb4fd7b871a48,Books,£47.82,£47.82,£0.00,In stock (20 available),0,£47.82,"[WICKED above her hipbone, GIRL across her hea..."
4,../../sapiens-a-brief-history-of-humankind_996...,Sapiens: A Brief History of Humankind,Five,4165285e1663650f,Books,£54.23,£54.23,£0.00,In stock (20 available),0,£54.23,[From a renowned historian comes a groundbreak...


# Ответы на некоторые вопросы по собранным данным

- У скольких книг отсутствует описание?

In [193]:
book_info_df['description'].isnull().sum()

1

In [192]:
book_info_df[book_info_df['description'].isnull()]

,link,name,rating,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews,price,description
160,../../the-bridge-to-consciousness-im-writing-t...,The Bridge to Consciousness: I'm Writing the B...,Three,efc3768127714ec3,Books,£32.00,£32.00,£0.00,In stock (15 available),0,£32.00,None


In [194]:
ans1 = book_info_df['description'].isnull().sum() 
ans1


1

- Сколько раз в данных встречается налог, больший нуля? 

In [132]:
ans2 = (book_info_df['Tax'] > 0).sum()
ans2

0

- Сколько раз рейтинг книги составлял пять звезд? 

In [133]:
ans3 = (book_info_df['rating'] == 'Five').sum()
ans3

37

- Какова средняя цена книг (без учета налога)? 

In [134]:
book_info_df['Price (excl. tax)'] = book_info_df['Price (excl. tax)'].apply(lambda w: float(w[1:]))
ans4 = book_info_df['Price (excl. tax)'].mean() 
ans4

34.79625